In [1]:
#GOGADA PRANATHI, 16369822

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
df = pd.read_csv("train.csv")
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


a)Impute Missing Values

In [5]:
print("Initial missing values count:")
print(df.isnull().sum())

Initial missing values count:
Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [6]:
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ['float64', 'int64']:
            # For numeric columns, we impute using the median.
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"Numeric column '{col}': Imputed missing values with median = {median_val}")
        else:
            # For non-numeric (categorical) columns, we impute using the mode.
            mode_val = df[col].mode()[0]
            df[col].fillna(mode_val, inplace=True)
            print(f"Categorical column '{col}': Imputed missing values with mode = '{mode_val}'")

Categorical column 'Mileage': Imputed missing values with mode = '18.9 kmpl'
Categorical column 'Engine': Imputed missing values with mode = '1197 CC'
Categorical column 'Power': Imputed missing values with mode = '74 bhp'
Numeric column 'Seats': Imputed missing values with median = 5.0
Categorical column 'New_Price': Imputed missing values with mode = '4.78 Lakh'


<ipython-input-6-37080f4fe6e7>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val, inplace=True)
<ipython-input-6-37080f4fe6e7>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

In [7]:
print("\nMissing values count after imputation:")
print(df.isnull().sum())


Missing values count after imputation:
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


Justification:
We first identified all columns with missing values to ensure data completeness. For numeric columns, we used the median to impute missing values, as it is robust against outliers. For categorical columns, we used the mode (most frequent value) to maintain consistency in categories. This method helps preserve the original distribution of the data while avoiding data loss. After imputation, all missing values were successfully handled, ensuring the dataset is ready for analysis or modeling.

b) Removing the units from some of the attributes

In [8]:

def extract_numeric(series):
    return pd.to_numeric(series.str.extract(r'([\d.]+)')[0], errors='coerce')

# Apply the function to clean columns
df['Mileage'] = extract_numeric(df['Mileage'].astype(str))
df['Engine'] = extract_numeric(df['Engine'].astype(str))
df['Power'] = extract_numeric(df['Power'].astype(str))
df['New_Price'] = extract_numeric(df['New_Price'].astype(str))  # Already string converted


In [9]:
print("\n✅ Cleaned Numeric Columns (After Removing Units):")
print(df[['Mileage', 'Engine', 'Power', 'New_Price']].head().to_string(index=False))



✅ Cleaned Numeric Columns (After Removing Units):
 Mileage  Engine  Power  New_Price
   19.67    1582 126.20       4.78
   13.00    1199  88.70       8.61
   20.77    1248  88.76       4.78
   15.20    1968 140.80       4.78
   23.08    1461  63.10       4.78


C) Changing the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value.

In [10]:
df_encoded = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], prefix=['Fuel', 'Trans'])


In [11]:
print("Data after one-hot encoding 'Fuel_Type' and 'Transmission':")
print(df_encoded.head())

Data after one-hot encoding 'Fuel_Type' and 'Transmission':
   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0              41000      First    19.67    1582  126.20    5.0       4.78   
1              46000      First    13.00    1199   88.70    5.0       8.61   
2              87000      First    20.77    1248   88.76    7.0       4.78   
3              40670     Second    15.20    1968  140.80    5.0       4.78   
4              86999      First    23.08    1461   63.10    5.0       4.78   

   Price  Fuel_Diesel  Fuel_Electric  Fuel_Petrol  Trans

d) Creating one more feature and add this column to the dataset

In [12]:
current_year = 2025
# Create  new column 'Age' by subtracting the 'Year' from the current year
df['Age'] = current_year - df['Year']

In [13]:
print("Data with the new 'Age' feature:")
print(df[['Year', 'Age']].head())

Data with the new 'Age' feature:
   Year  Age
0  2015   10
1  2011   14
2  2012   13
3  2013   12
4  2013   12


In [14]:

print(df.head())

   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage  Engine  \
0              41000    Diesel       Manual      First    19.67    1582   
1              46000    Petrol       Manual      First    13.00    1199   
2              87000    Diesel       Manual      First    20.77    1248   
3              40670    Diesel    Automatic     Second    15.20    1968   
4              86999    Diesel       Manual      First    23.08    1461   

    Power  Seats  New_Price  Price  Age  
0  126.20    5.0       4.78  12.50   10  
1   88.70    5.0       8.61   4.50   14  
2   88.7

e) Performing select, filter, rename, mutate, arrange and summarize with group by operations

In [15]:
#1. SELECT
df_selected = df[['Name', 'Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Price']]
print("Selected Columns:")
print(df_selected.head())

Selected Columns:
                               Name  Year  Kilometers_Driven Fuel_Type  \
0  Hyundai Creta 1.6 CRDi SX Option  2015              41000    Diesel   
1                      Honda Jazz V  2011              46000    Petrol   
2                 Maruti Ertiga VDI  2012              87000    Diesel   
3   Audi A4 New 2.0 TDI Multitronic  2013              40670    Diesel   
4            Nissan Micra Diesel XV  2013              86999    Diesel   

  Transmission  Price  
0       Manual  12.50  
1       Manual   4.50  
2       Manual   6.00  
3    Automatic  17.74  
4       Manual   3.50  


In [16]:
#2. FILTER
df_filtered = df_selected[(df_selected['Year'] > 2010) & (df_selected['Price'] <= 10.0)]
print("\nFiltered Rows (Year > 2010 and Price <= 10):")
print(df_filtered.head())



Filtered Rows (Year > 2010 and Price <= 10):
                                  Name  Year  Kilometers_Driven Fuel_Type  \
1                         Honda Jazz V  2011              46000    Petrol   
2                    Maruti Ertiga VDI  2012              87000    Diesel   
4               Nissan Micra Diesel XV  2013              86999    Diesel   
6  Volkswagen Vento Diesel Comfortline  2013              64430    Diesel   
7       Tata Indica Vista Quadrajet LS  2012              65932    Diesel   

  Transmission  Price  
1       Manual   4.50  
2       Manual   6.00  
4       Manual   3.50  
6       Manual   5.20  
7       Manual   1.95  


In [17]:
# 3. RENAME: Rename columns for clarity
df_renamed = df_filtered.rename(columns={'Kilometers_Driven': 'Kms_Driven', 'Price': 'Sale_Price'})
print("\nData after Renaming Columns:")
print(df_renamed.head())


Data after Renaming Columns:
                                  Name  Year  Kms_Driven Fuel_Type  \
1                         Honda Jazz V  2011       46000    Petrol   
2                    Maruti Ertiga VDI  2012       87000    Diesel   
4               Nissan Micra Diesel XV  2013       86999    Diesel   
6  Volkswagen Vento Diesel Comfortline  2013       64430    Diesel   
7       Tata Indica Vista Quadrajet LS  2012       65932    Diesel   

  Transmission  Sale_Price  
1       Manual        4.50  
2       Manual        6.00  
4       Manual        3.50  
6       Manual        5.20  
7       Manual        1.95  


In [18]:
# 4. MUTATE: Create a new column (mutate equivalent)
df_renamed['Age'] = 2025 - df_renamed['Year']
print("\nData after Adding 'Age' Column:")
print(df_renamed.head())


Data after Adding 'Age' Column:
                                  Name  Year  Kms_Driven Fuel_Type  \
1                         Honda Jazz V  2011       46000    Petrol   
2                    Maruti Ertiga VDI  2012       87000    Diesel   
4               Nissan Micra Diesel XV  2013       86999    Diesel   
6  Volkswagen Vento Diesel Comfortline  2013       64430    Diesel   
7       Tata Indica Vista Quadrajet LS  2012       65932    Diesel   

  Transmission  Sale_Price  Age  
1       Manual        4.50   14  
2       Manual        6.00   13  
4       Manual        3.50   12  
6       Manual        5.20   12  
7       Manual        1.95   13  


In [19]:
# 5. ARRANGE: Sort the dataset based on a column
df_sorted = df_renamed.sort_values(by='Sale_Price')
print("\nData Sorted by 'Sale_Price':")
print(df_sorted.head())


Data Sorted by 'Sale_Price':
                          Name  Year  Kms_Driven Fuel_Type Transmission  \
1660              Tata Nano Lx  2011       65000    Petrol       Manual   
4821          Tata Nano STD SE  2012       80000    Petrol       Manual   
2254              Tata Nano XT  2011       50000    Petrol       Manual   
4813  Tata Indica V2 DLE BSIII  2011       96000    Diesel       Manual   
340     Chevrolet Spark 1.0 LS  2011       21000    Petrol       Manual   

      Sale_Price  Age  
1660        0.44   14  
4821        0.53   13  
2254        0.75   14  
4813        0.91   14  
340         0.95   14  


In [21]:
# 6. SUMMARIZE  Group data and aggregate
summary = df.groupby('Fuel_Type').agg(
    Avg_Sale_Price=('Price', 'mean'),
    Avg_Age=('Year', lambda x: 2025 - x.mean()),  # Calculate average car age
    Count=('Fuel_Type', 'count')
).reset_index()
print("\nSummary Statistics by Fuel_Type:")
print(summary)


Summary Statistics by Fuel_Type:
  Fuel_Type  Avg_Sale_Price    Avg_Age  Count
0    Diesel       12.960686  11.209111   3161
1  Electric       12.875000  11.500000      2
2    Petrol        5.756688  11.954918   2684
